#### IDs present on this list are from 07/06/2023
#### https://api.steampowered.com/ISteamApps/GetAppList/v2/ 

In [1]:
import requests
import pandas as pd
import json

#### Get app_ids

In [2]:

url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"  # URL with the list of apps
response = requests.get(url) # Send a GET request to the URL
json_data = response.json() # Convert the response to JSON format

id_list = [] # Empty list to store the app IDs

if response.status_code == 200: # if the request was successful
    for item in json_data["applist"]["apps"]:
        # Iterate over each app in the "apps" list within the "applist" dictionary
        id_list.append(item["appid"]) # Extract the "appid" and add it to the id_list
id_list.sort() # Sort the app IDs in ascending order

#### App_Details

In [3]:
app_data_list = []  # empty list for fetched data

for ids in id_list:
    try: #error handling
        url = f"https://store.steampowered.com/api/appdetails/?appids={ids}"  # iterate through all IDs fetched from the code above
        response = requests.get(url)
        if response.status_code == 200:
            json_data = response.json()
            for key, value in json_data.items():
                app_data = value.get('data')
                if app_data is not None and app_data.get('type') == 'game':  # only want the games
                    app_data_list.append(app_data) # add the game data to the app_data_list
    except Exception as e: #some ids are throwing errors so lets try to isolate them
        print(f"Error fetching data for ID: {ids}. Error message: {e}")
        continue

df = pd.json_normalize(app_data_list)  #Create a DataFrame from the app_data_list

Error fetching data for ID: 1024100. Error message: HTTPSConnectionPool(host='store.steampowered.com', port=443): Max retries exceeded with url: /api/appdetails/?appids=1024100 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
Error fetching data for ID: 1024460. Error message: HTTPSConnectionPool(host='store.steampowered.com', port=443): Max retries exceeded with url: /api/appdetails/?appids=1024460 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000253E08C1F10>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))
Error fetching data for ID: 1024600. Error message: HTTPSConnectionPool(host='store.steampowered.com', port=443): Max retries exceeded with url: /api/appdetails/?appids=1024600 (Caused by NewConnectionError('<urllib

#### lets remove most col we know we don't want

In [ ]:
columns_to_exclude = ['header_image', 'capsule_image', 'capsule_imagev5', 'background', 'support_info.url', 'support_info.email', 'metacritic.url'] #columns to be excluded
df = df.drop(columns=columns_to_exclude) 

In [13]:
df.to_excel("data/steam_orignal_urls_removed.xlsx") # store the data in an excel file